PWC-Net-small model finetuning (with cyclical learning rate schedule)
=======================================================

In this notebook we:
- Use a small model (no dense or residual connections), 6 level pyramid, uspample level 2 by 4 as the final flow prediction
- Train the PWC-Net-small model on a mix of the `FlyingChairs` and `FlyingThings3DHalfRes` dataset using a Cyclic<sub>short</sub> schedule of our own
- Let the Cyclic<sub>short</sub> schedule oscillate between `2e-05` and `1e-06` for 200,000 steps
- Switch to the "robust" loss described in the paper, instead of the "multiscale" loss used during training

Below, look for `TODO` references and customize this notebook based on your own needs.

## Reference

[2018a]<a name="2018a"></a> Sun et al. 2018. PWC-Net: CNNs for Optical Flow Using Pyramid, Warping, and Cost Volume. [[arXiv]](https://arxiv.org/abs/1709.02371) [[web]](http://research.nvidia.com/publication/2018-02_PWC-Net%3A-CNNs-for) [[PyTorch (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/PyTorch) [[Caffe (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/Caffe)

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
"""
pwcnet_finetune.ipynb

PWC-Net model finetuning.

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)

Tensorboard:
    [win] tensorboard --logdir=E:\\repos\\tf-optflow\\tfoptflow\\pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned
    [ubu] tensorboard --logdir=/media/EDrive/repos/tf-optflow/tfoptflow/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned
"""
from __future__ import absolute_import, division, print_function
import sys
from copy import deepcopy
from dataset_mpisintel import MPISintelDataset
from dataset_base import _DEFAULT_DS_TUNE_OPTIONS
from dataset_flyingchairs import FlyingChairsDataset
from dataset_flyingthings3d import FlyingThings3DHalfResDataset
from dataset_kitti import KITTIDataset
from dataset_mixer import MixedDataset
from model_pwcnet import ModelPWCNet, _DEFAULT_PWCNET_FINETUNE_OPTIONS

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Vol0/user/f.konokhov/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-788693d64ca1>", line 17, in <module>
    from dataset_mpisintel import MPISintelDataset
  File "/Vol0/user/f.konokhov/tfoptflow/tfoptflow/dataset_mpisintel.py", line 13, in <module>
    from sklearn.model_selection import train_test_split
  File "/Vol0/user/f.konokhov/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/__init__.py", line 19, in <module>
    from ._validation import cross_val_score
  File "/Vol0/user/f.konokhov/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py", line 31, in <module>
    from ..metrics.scorer import check_scoring, _check_multimetric_scoring
  File "/Vol0/user/f.konokhov/miniconda3/lib/python3.6/site-packages/sklearn/metrics/__init__.py", line 7, in <module>
    from .ranking impo

KeyboardInterrupt: 

In [ ]:
!nvidia-smi

## TODO: Set this first!

In [ ]:
# TODO: You MUST set dataset_root to the correct path on your machine!
if sys.platform.startswith("win"):
    _DATASET_ROOT = 'E:/datasets/'
else:
    _DATASET_ROOT = '/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/flow_train/' #'/Vol1/dbstore/datasets/sintel/'

    
# TODO: You MUST adjust the settings below based on the number of GPU(s) used for training
# Set controller device and devices
# A one-gpu setup would be something like controller='/device:GPU:0' and gpu_devices=['/device:GPU:0']
# Here, we use a dual-GPU setup, as shown below
# gpu_devices = ['/device:GPU:0', '/device:GPU:1']
# controller = '/device:CPU:0'
gpu_devices = ['/device:GPU:1']
controller = '/device:GPU:1'

# TODO: You MUST adjust this setting below based on the amount of memory on your GPU(s)
# Batch size
batch_size = 2

# Finetune on `sintel` mix

## Load the dataset

In [ ]:
# TODO: You MUST set the batch size based on the capabilities of your GPU(s) 
#  Load train dataset
ds_opts = deepcopy(_DEFAULT_DS_TUNE_OPTIONS)
ds_opts['in_memory'] = False                          # Too many samples to keep in memory at once, so don't preload them
ds_opts['aug_type'] = 'basic'                         # Apply all supported augmentations
ds_opts['batch_size'] = batch_size * len(gpu_devices) # Use a multiple of 8; here, 16 for dual-GPU mode (Titan X & 1080 Ti)
#ds1 = FlyingChairsDataset(mode='train_with_val', ds_root=_FLYINGCHAIRS_ROOT, options=ds_opts)
ds_opts['type'] = 'npy'

#ds2 = FlyingThings3DHalfResDataset(mode='train_with_val', ds_root=_FLYINGTHINGS3DHALFRES_ROOT, options=ds_opts)
#ds = MixedDataset(mode='train_with_val', datasets=[ds1, ds2], options=ds_opts)

In [ ]:
ds_opts['crop_preproc'] = (320,960)

In [7]:
# ds = MPISintelDataset(mode='train_with_val', ds_root=_MPISINTEL_ROOT, options=ds_opts)

In [8]:
ds = KITTIDataset(mode = 'train_with_val', ds_root=_DATASET_ROOT, options=ds_opts)

In [9]:
# Display dataset configuration
ds.print_config()


Dataset Configuration:
  verbose              False
  in_memory            False
  crop_preproc         (320, 960)
  scale_preproc        None
  type                 npy
  tb_test_imgs         False
  random_seed          1969
  val_split            0.03
  aug_type             basic
  aug_labels           True
  fliplr               0.5
  flipud               0.5
  translate            (0.5, 0.05)
  scale                (0.5, 0.05)
  batch_size           2
  mode                 train_with_val
  train size           2201
  val size             69


## Configure the finetuning

In [ ]:
# Start from the default options
nn_opts = deepcopy(_DEFAULT_PWCNET_FINETUNE_OPTIONS)
nn_opts['verbose'] = True
nn_opts['ckpt_path'] = './pwcnet-lg-6-2-cyclic-sintel_final_finetuned/pwcnet.ckpt-43000'
nn_opts['ckpt_dir'] = './pwcnet-lg-6-2-cyclic-KITTI_like_torch/'
nn_opts['batch_size'] = 8
nn_opts['x_shape'] = [2, ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 3]
nn_opts['y_shape'] = [ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 2]
nn_opts['use_tf_data'] = False #True # Use tf.data reader
nn_opts['gpu_devices'] = gpu_devices
nn_opts['controller'] = controller

# Use the PWC-Net-small model in quarter-resolution mode
nn_opts['use_dense_cx'] = True
nn_opts['use_res_cx'] = True
nn_opts['pyr_lvls'] = 6
nn_opts['flow_pred_lvl'] = 2

# Robust loss as described doesn't work, so try the following:
nn_opts['loss_fn'] = 'loss_multiscale' # 'loss_multiscale' # 'loss_robust' # 'loss_robust'
nn_opts['q'] = 1. # 0.4 # 1. # 0.4 # 1.
nn_opts['epsilon'] = 0. # 0.01 # 0. # 0.01 # 0.

In [ ]:
# Set the learning rate schedule. This schedule is for a single GPU using a batch size of 8.
# Below,we adjust the schedule to the size of the batch and the number of GPUs.
nn_opts['lr_policy'] = 'multisteps'
nn_opts['init_lr'] = 1e-05
nn_opts['lr_boundaries'] = [80000, 120000, 160000, 200000]
nn_opts['lr_values'] = [1e-05, 5e-06, 2.5e-06, 1.25e-06, 6.25e-07]
nn_opts['max_steps'] = 200000

# Below,we adjust the schedule to the size of the batch and our number of GPUs (2).
nn_opts['max_steps'] = int(nn_opts['max_steps'] * 8 / 8)
nn_opts['cyclic_lr_stepsize'] = int(nn_opts['cyclic_lr_stepsize'] * 8 / 8)

In [ ]:
# Instantiate the model and display the model configuration
nn = ModelPWCNet(mode='train_with_val', options=nn_opts, dataset=ds)
nn.print_config()

Building model...
... model built.
Configuring training ops...
... training ops configured.
Initializing from pre-trained model at ./pwcnet-lg-6-2-cyclic-sintel_final_finetuned/pwcnet.ckpt-43000 for finetuning...

INFO:tensorflow:Restoring parameters from ./pwcnet-lg-6-2-cyclic-sintel_final_finetuned/pwcnet.ckpt-43000
... model initialized

Model Configuration:
  verbose                True
  ckpt_path              ./pwcnet-lg-6-2-cyclic-sintel_final_finetuned/pwcnet.ckpt-43000
  ckpt_dir               ./pwcnet-lg-6-2-cyclic-KITTI_like_torch/
  max_to_keep            10
  x_dtype                <dtype: 'float32'>
  x_shape                [2, 320, 960, 3]
  y_dtype                <dtype: 'float32'>
  y_shape                [320, 960, 2]
  train_mode             fine-tune
  adapt_info             None
  sparse_gt_flow         False
  display_step           100
  snapshot_step          1000
  val_step               1000
  val_batch_size         -1
  tb_val_imgs            top_flow
  tb_te

## Finetune the model

In [ ]:
# Train the model
nn.train()

Start finetuning...
2019-06-04 13:53:33 Iter 100 [Train]: loss=133.89, epe=3.00, lr=0.000010, samples/sec=4.9, sec/step=1.641, eta=3 days, 19:06:16
2019-06-04 13:57:22 Iter 200 [Train]: loss=112.99, epe=2.46, lr=0.000010, samples/sec=5.6, sec/step=1.419, eta=3 days, 6:44:13
2019-06-04 14:01:10 Iter 300 [Train]: loss=115.27, epe=2.52, lr=0.000010, samples/sec=5.6, sec/step=1.424, eta=3 days, 6:59:37
2019-06-04 14:05:03 Iter 400 [Train]: loss=105.21, epe=2.29, lr=0.000010, samples/sec=5.6, sec/step=1.416, eta=3 days, 6:31:28
2019-06-04 14:08:53 Iter 500 [Train]: loss=105.23, epe=2.32, lr=0.000010, samples/sec=5.6, sec/step=1.417, eta=3 days, 6:30:23
2019-06-04 14:12:39 Iter 600 [Train]: loss=97.66, epe=2.10, lr=0.000010, samples/sec=5.6, sec/step=1.419, eta=3 days, 6:34:13
2019-06-04 14:16:25 Iter 700 [Train]: loss=101.83, epe=2.27, lr=0.000010, samples/sec=5.6, sec/step=1.422, eta=3 days, 6:43:19
2019-06-04 14:20:18 Iter 800 [Train]: loss=97.25, epe=2.16, lr=0.000010, samples/sec=5.6, s

## Training log

Here are the training curves for the run above:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/loss.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/epe.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/lr.png)

Here are the predictions issued by the model for a few validation samples:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val1.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val2.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val3.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val4.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val5.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val6.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val7.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val8.png)